In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

import numpy as np
import pandas as pd
import runReef as reef

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Present-day distribution of coral reefs in the tropics

In [ ]:
coral = pd.read_csv("data/Coral.csv",index_col='ID')
coral.head(3)

In [ ]:
lons = coral['LON'].values
lats = coral['LAT'].values

proj = ccrs.Mollweide(180)
fig = plt.figure(figsize=(10,8))
ax = plt.axes(projection=proj)
ax.stock_img()
ax.coastlines(linewidth=1)
pts = ax.scatter(lons, lats, s=30, c='r',
           edgecolor='black',alpha=0.75, transform=ccrs.PlateCarree())
plt.show()
fig.clear()
plt.close(fig)

# Reef 2D

The variety of coral reefs morphologies highlights their sensitivities to several forcings; fossil reefs stack in sequences that are accordingly diverse. In order to understand their genesis and architectures, we will use a numerical approach, accounting for sea level oscillations, vertical land motion, initial slope, wave erosion, and reef growth.

**We will explore the formation of coral reef in 2D under different sea-level and tectonic forcing conditions.**

> Husson, L., Pastier, A.-M., Pedoja, K., Elliot, M., Paillard, D., Authemayou, C., et al. (2018). Reef carbonate productivity during Quaternary sea level oscillations. Geochemistry, Geophysics, Geosystems, 19. https://doi.org/10.1002/2017GC007335

<img src="imgs/reef2d.png" alt="model" width="700"/>


Model output after a 10-kyr simulation. Gray domain is the basement (initially having a linear surface), blue line is sea level, pink units are reef bodies, color-coded by age every 1 kyr. Shaded units are sedimentary layers. G, E, and S denote reef growth rate, erosion rate, and sedimentation rate, respectively. 

# Setting up the model

The model requires several initial conditions that are listed below:

In [ ]:
# RSL variations file
rsl = 'stationary.dat' 
# Slope (%)
slp = '10' 
# Maximum reef growth rate (mm/yr)
rg = '18'
# Maximum reef growth depth (m)
rd = '30'
# Optimal depth for vertical gradient (m)
vg = '1' 
# Optimal depth for horizontal gradient (m)
hg = '3' 
# Maximum depth for wave erosion (m)
wd = '3'
# Uplift rate (mm/yr)
ur = '0.0'
# Eroded volume (mm3/yr)
ev = '100'
# Temporal increment (yr)
dt = '1000'
# Reef construction or not ?
rc = '.true.'
# Wave erosion or not ?
we = '.true.'

lines = [rsl, slp, rg, rd, vg, hg, wd, ur, ev, dt, rc, we]
with open('param0.dat', 'w') as f:
    f.write('\n'.join(lines))

## Fringing reef

The most common type of reef is a **fringing reef**. They grow directly from land, creating a border along the shoreline.

<img src="imgs/fringing.jpg" alt="fringing" width="500"/>

In [ ]:
foutput = 'model'
reef.runReefModel(foutput)
outdata = reef.readReefOutput(foutput)
reef.plotReef(outdata,zrange=[-52,20])

# Imposing a tectonic scenario

## Barrier reef

Although they also form parallel to the land, **barrier reefs** grow further out to sea and are separated from the shore by a lagoon.

<img src="imgs/barrier.jpg" alt="barrier" width="700"/>

Let's assume the same input as before but this time with a subisdence rate of 1 mm/yr:

In [ ]:
# Uplift rate (mm/yr)
ur = '-1.0'
# Wave erosion or not ?
we = '.false.'

lines = [rsl, slp, rg, rd, vg, hg, wd, ur, ev, dt, rc, we]
with open('param0.dat', 'w') as f:
    f.write('\n'.join(lines)) 

reef.runReefModel(foutput)
outdata = reef.readReefOutput(foutput)
reef.plotReef(outdata,zrange=[-52,15])

## Impact of karstification and wave erosion

We will now impose a uniform uplift and activate the wave erosion:

In [ ]:
# Uplift rate (mm/yr)
ur = '1.' 
# Wave erosion or not ?
we = '.true.'

lines = [rsl, slp, rg, rd, vg, hg, wd, ur, ev, dt, rc, we]
with open('param0.dat', 'w') as f:
    f.write('\n'.join(lines)) 
reef.runReefModel(foutput)
outdata = reef.readReefOutput(foutput)
reef.plotReef(outdata,zrange=[-42,25])

# Defining a simple sinusoidal sea-level curve

Let's create a sea-level curve to test how it impacts coral reef evolution.

In [ ]:
def makeSinSL(cyc,res,amp,slfile):
    
    # Number of sine cycles
    cycles = cyc
    # Time range (kyr)
    resolution = res 
    # Sea level amplitude (m)
    amplitude = amp

    length = np.pi * 2 * cycles
    my_wave = -np.sin(np.arange(0, length, length / resolution))*amplitude

    data = {'time':np.flip(np.arange(resolution)),
            'sea':my_wave
            }
    dfsl = pd.DataFrame(data)
    dfsl.to_csv('data/sin.dat', index=False, sep=' ',header=False)
    
    fig, ax1 = plt.subplots(1,1,figsize=(6,2))
    ax1.plot(-dfsl['time'], dfsl['sea'], '-', color='navy', zorder=3, lw=2)
    ax1.set_ylabel('Sea-level (m)', fontsize=9, weight='bold')
    ax1.set_xlabel('Time (kyr)', fontsize=9, weight='bold')
    ax1.tick_params(axis="x", labelsize=8)
    ax1.tick_params(axis="y", labelsize=8)
    plt.show()

    return 

makeSinSL(cyc=2,res=21,amp=5.,slfile='data/sin.dat')

## Impact of sea-level of reef evolution

We put back simular parameters as in the first example (no tectonics), but this time we impose the new sea-level curve:

In [ ]:
# RSL variations file
rsl = 'sin.dat' 
# Optimal depth for horizontal gradient (m)
hg = '6'
# Uplift rate (mm/yr)
ur = '0.0'
# Wave erosion or not ?
we = '.false.'

lines = [rsl, slp, rg, rd, vg, hg, wd, ur, ev, dt, rc, we]
with open('param0.dat', 'w') as f:
    f.write('\n'.join(lines)) 
reef.runReefModel(foutput)
outdata = reef.readReefOutput(foutput)
reef.plotReef(outdata,zrange=[-52,15])

## Combining tectonic and sea-level

Adding a subsidence history with the sea-level fluctuations:

In [ ]:
# Uplift rate (mm/yr)
ur = '-1.0'
# Wave erosion or not ?
we = '.false.'

lines = [rsl, slp, rg, rd, vg, hg, wd, ur, ev, dt, rc, we]
with open('param0.dat', 'w') as f:
    f.write('\n'.join(lines)) 
reef.runReefModel(foutput)
outdata = reef.readReefOutput(foutput)
reef.plotReef(outdata,zrange=[-52,15])

## Perched reef terraces

> Pastier, A.-M., Husson, L., Pedoja, K., Bézos, A., Authemayou, C., Arias-Ruiz, C., & Cahyarini, S. Y. (2019).Genesis and architecture of sequences of Quaternary coral reef terraces: Insights from numerical models. Geochemistry, Geophysics, Geosystems, 20, 4248–4272. https://doi. org/10.1029/2019GC008239


<img src="imgs/terraces.png" alt="terraces" width="800"/>

_(a) Sequences of emerged reef terraces, Sampolawa Bay, SE Sulawesi (Fortuin et al., 1990; Pedoja et al., 2018). White dashed lines delineate uplifted terraces. (b) Close-up showing nested narrow terraces and fossil notches. The active notch is carving a retrograding cliff in earlier reefal units, which progressively erases earlier sea level markers. The erratic block in the forefront witnesses this retrogradation, while the preservation of fossil notches attests for moderate marine erosion._

In [ ]:
proj2 = ccrs.Mercator()
fig = plt.figure(figsize=(8,6))
ax = plt.axes(projection=proj2)
ax.coastlines(linewidth=1)
pts = ax.scatter(lons, lats, s=30, c='r',
           edgecolor='black',alpha=0.75, transform=ccrs.PlateCarree())
ax.set_extent([85, 140, -20, 20], crs=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.OCEAN)
plt.show()
fig.clear()
plt.close(fig)

We will combine wave erosion, uplift and sealevel change:

In [ ]:
# Uplift rate (mm/yr)
ur = '1.0'
# Wave erosion or not ?
we = '.true.'

lines = [rsl, slp, rg, rd, vg, hg, wd, ur, ev, dt, rc, we]
with open('param0.dat', 'w') as f:
    f.write('\n'.join(lines)) 
reef.runReefModel(foutput)
outdata = reef.readReefOutput(foutput)
reef.plotReef(outdata,zrange=[-52,35])